<a href="https://colab.research.google.com/drive/1yfN9zoB8WNNBjH_RhMnqp7qFwACnCA_g?usp=sharing" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

In [1]:
# Install required dependencies
!pip install -U crewai


In [38]:
# Section 1: Augmented LLM (Tool Use) - CrewAI using Groq's deepseek-r1-distill-llama-70b
# This section sets up an agent that solves math expressions using tools only.

# Set Groq API key and endpoint
import os
import time
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

# Import CrewAI components
from crewai.tools import tool
from crewai import Agent, Task, Crew, Process

# Define tools with required docstrings
@tool("Multiply")
def multiply(a: int, b: int) -> int:
    """Multiply two integers and return the result."""
    return a * b

@tool("Divide")
def divide(a: int, b: int) -> float:
    """Divide a by b and return the result."""
    return a / b

# Define the tool-using agent with DeepSeek LLaMA 70B
agent = Agent(
    name="ToolCalcAgent",
    role="Math solver using tools",
    goal="Use tools like Multiply and Divide to evaluate expressions.",
    backstory="This agent is not allowed to do mental math. It must use tools only.",
    llm="groq/deepseek-r1-distill-llama-70b",
    tools=[multiply, divide],
    system_prompt="Use the available tools to solve the given math problem. Do not calculate anything mentally.",
    allow_delegation=False,
    verbose=False
)

# Define the task
task = Task(
    name="Evaluate Expression",
    description="Use tools to solve the expression: '{expr}'",
    expected_output="The numeric result.",
    agent=agent
)

# Create a crew to manage agent-task workflow
crew = Crew(
    agents=[agent],
    tasks=[task],
    process=Process.sequential,
    max_rounds=4
)

# Run test expressions and print final results only
expressions = [
    "8 * 4",
    "100 / 5",
    "10 / 2 * 3",

]

for expr in expressions:
    result = crew.kickoff({"expr": expr})
    print(f"{expr} = {result}")
    time.sleep(6)  # Wait few seconds to reset token limits


8 * 4 = 32
100 / 5 = 20.0
10 / 2 * 3 = 15.0


In [39]:
# Section 2: Prompt Chaining - CrewAI using groq/deepseek-r1-distill-llama-70b

# Set Groq API credentials
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

# Import CrewAI components
from crewai import Agent, Task, Crew, Process

# Define the first agent to extract three bullet-point insights
insight_agent = Agent(
    name="InsightAgent",
    role="Insight Extractor",
    goal="Identify three key points about a topic",
    backstory="A research assistant who summarizes topics into key takeaways.",
    llm="groq/deepseek-r1-distill-llama-70b",
    tools=[],
    system_prompt="Extract exactly three bullet-point insights about the given topic.",
    verbose=False
)

# Define the second agent to write a paragraph from those insights
writer_agent = Agent(
    name="WriterAgent",
    role="Paragraph Writer",
    goal="Convert insights into a well-written paragraph",
    backstory="A writer who explains ideas based on bullet points.",
    llm="groq/deepseek-r1-distill-llama-70b",
    tools=[],
    system_prompt="Write a paragraph based on the previously extracted bullet points.",
    verbose=False
)

# Define the first task: extract insights
task1 = Task(
    name="Extract Insights",
    description="List three key insights about '{topic}'.",
    expected_output="Three bullet points.",
    agent=insight_agent
)

# Define the second task: write a paragraph from insights
task2 = Task(
    name="Write Paragraph",
    description="Use the bullet points to write a paragraph.",
    expected_output="A single coherent paragraph.",
    agent=writer_agent
)

# Create the Crew with sequential process
crew = Crew(
    agents=[insight_agent, writer_agent],
    tasks=[task1, task2],
    process=Process.sequential,
    max_rounds=3
)

# Run the pattern on a test topic
result = crew.kickoff(inputs={"topic": "blockchain technology"})
print("Final Paragraph:\n", result)


Final Paragraph:
 <think>
Okay, I need to convert the bullet points into a well-written paragraph. Let me start by understanding each point. The first point talks about decentralization and security. I should explain how the decentralized nature and cryptographic linking make blockchain secure. 

The second point is about versatility and applications. I need to mention examples like supply chain and smart contracts to show its wide use beyond finance.

The third point is about immutability and trust. I should highlight how the data cannot be altered once recorded, which builds trust in systems like voting.

I should connect these points smoothly, ensuring the paragraph flows naturally. I'll start with an introduction to blockchain, then discuss each key insight in order, and conclude with its importance. Let me make sure each sentence transitions well and the paragraph is coherent.
</think>

Blockchain technology, known for its role in cryptocurrencies, offers three key insights. First

In [40]:
# Section 3: Parallelization - CrewAI using groq/deepseek-r1-distill-llama-70b (one cell)

# Set Groq API credentials
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

# Import CrewAI
from crewai import Agent, Task, Crew, Process

# Define the model to use
llm_model = "groq/deepseek-r1-distill-llama-70b"

# Define agent 1: summarizes the topic
summarizer = Agent(
    name="SummarizerAgent",
    role="Summary Generator",
    goal="Provide a concise two-sentence summary of a topic",
    backstory="An expert in boiling down information to its core points.",
    llm=llm_model,
    tools=[],
    system_prompt="Summarize the topic in exactly two sentences.",
    verbose=False
)

# Define agent 2: creates an analogy
analogist = Agent(
    name="AnalogistAgent",
    role="Analogy Creator",
    goal="Generate an analogy that simplifies the topic",
    backstory="A creative mind who translates concepts into metaphors.",
    llm=llm_model,
    tools=[],
    system_prompt="Create a relatable analogy that explains the topic.",
    verbose=False
)

# Define agent 3: provides a fun fact
factoid = Agent(
    name="FactoidAgent",
    role="Fun Fact Provider",
    goal="Provide an interesting fact about the topic",
    backstory="Finds surprising details people don’t know.",
    llm=llm_model,
    tools=[],
    system_prompt="Share a surprising or little-known fact about the topic.",
    verbose=False
)

# Define the final agent to merge all three outputs
merger = Agent(
    name="MergerAgent",
    role="Output Merger",
    goal="Combine the outputs into a single cohesive paragraph",
    backstory="An editor who stitches together diverse ideas.",
    llm=llm_model,
    tools=[],
    system_prompt="Merge the summary, analogy, and fact into one paragraph.",
    verbose=False
)

# Define tasks for summary, analogy, fact — async execution
task_summary = Task(
    name="SummarizeTopic",
    description="Summarize the topic '{topic}' in two sentences.",
    expected_output="Two-sentence summary.",
    agent=summarizer,
    async_execution=True
)

task_analogy = Task(
    name="AnalogyForTopic",
    description="Create a metaphor or analogy for the topic '{topic}'.",
    expected_output="One analogy.",
    agent=analogist,
    async_execution=True
)

task_fact = Task(
    name="FunFactAboutTopic",
    description="Provide a fun or surprising fact about the topic '{topic}'.",
    expected_output="One fun fact.",
    agent=factoid,
    async_execution=True
)

# Define the final merge task
task_merge = Task(
    name="MergeResults",
    description="Combine the summary, analogy, and fun fact into one paragraph.",
    expected_output="A merged paragraph.",
    agent=merger
)

# Create the crew with all agents and tasks
crew = Crew(
    agents=[summarizer, analogist, factoid, merger],
    tasks=[task_summary, task_analogy, task_fact, task_merge],
    process=Process.sequential,
    max_rounds=3
)

# Run the crew on a test topic
result = crew.kickoff(inputs={"topic": "machine learning"})
print("Final Combined Output:\n", result)


Final Combined Output:
 <think>
Alright, I need to combine the summary, analogy, and fun fact into one cohesive paragraph. Let me start by looking at each part.

The summary is about machine learning being a subset of AI that learns from data without explicit programming, used in various fields. The analogy compares it to an apprentice learning from a master. The fun fact is about machine learning creating art through projects like DeepDream.

I need to merge these into a single paragraph. Maybe start with the summary, then the analogy, and end with the fun fact. I should ensure smooth transitions between each part.

Let me structure it: first, define machine learning, then explain how it's like an apprentice, and finally add the surprising fact about art. That should flow well.

I also need to make sure it's concise and each part connects logically. Avoid making it too lengthy, but still informative.

Okay, putting it all together now.
</think>

Machine learning is a subset of artific

In [19]:
# Section 4: Routing - CrewAI using groq/deepseek-r1-distill-llama-70b

# Set Groq API credentials
import os
import time

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

# Import CrewAI
from crewai import Agent, Task, Crew, Process

# Define model to use
llm_model = "groq/deepseek-r1-distill-llama-70b"

# Define router agent
router_agent = Agent(
    name="RouterAgent",
    role="Classifier",
    goal="Determine if a request is asking for a summary, analogy, or fun fact.",
    backstory="Understands input types and routes them to the correct expert.",
    llm=llm_model,
    tools=[],
    system_prompt="Label the user query as 'summary', 'analogy', or 'fun_fact'. Return only one word.",
    verbose=False
)

# Define specialist agents
summary_agent = Agent(
    name="SummaryAgent",
    role="Summarizer",
    goal="Summarize the query in two sentences.",
    backstory="A concise summarizer.",
    llm=llm_model,
    tools=[],
    system_prompt="Write a two-sentence summary of the query, be brief.",
    verbose=False
)

analogy_agent = Agent(
    name="AnalogyAgent",
    role="Analogist",
    goal="Create a relatable analogy.",
    backstory="Explains abstract ideas through metaphor.",
    llm=llm_model,
    tools=[],
    system_prompt="Write one brief analogy that explains the query.",
    verbose=False
)

fact_agent = Agent(
    name="FactAgent",
    role="Fact Finder",
    goal="Share a surprising fact.",
    backstory="Knows strange and fun trivia.",
    llm=llm_model,
    tools=[],
    system_prompt="Give one fun fact related to the query, be brief.",
    verbose=False
)

# Define classification task
classify_task = Task(
    name="ClassifyQuery",
    description="Classify '{query}' as summary, analogy, or fun_fact.",
    expected_output="One of: summary, analogy, fun_fact",
    agent=router_agent
)

# Define tasks for all possible routes
task_summary = Task(
    name="WriteSummary",
    description="Write a two-sentence summary of '{query}'. Keep it brief.",
    expected_output="Summary text (2 sentences).",
    agent=summary_agent
)

task_analogy = Task(
    name="WriteAnalogy",
    description="Write one analogy for '{query}'. Keep it short.",
    expected_output="Analogy text.",
    agent=analogy_agent
)

task_fact = Task(
    name="WriteFunFact",
    description="Write one fun fact related to '{query}'. Keep it brief.",
    expected_output="Fact text.",
    agent=fact_agent
)

# Create the crew with all agents and tasks
crew = Crew(
    agents=[router_agent, summary_agent, analogy_agent, fact_agent],
    tasks=[classify_task, task_summary, task_analogy, task_fact],
    process=Process.sequential,
    max_rounds=3
)

# Run routing on a sample query and handle output sequentially
query = "Explain how black holes form"
result = crew.kickoff(inputs={"query": query})

# Add sleep to avoid hitting rate limits
time.sleep(2)  # Adding a 2-second delay to handle Groq's rate limits

# Determine which route was taken based on classification
label = classify_task.output.raw.strip().lower()

# Handling each route’s output based on classification
if label == "summary":
    final_output = task_summary.output.raw
elif label == "analogy":
    final_output = task_analogy.output.raw
elif label == "fun_fact":
    final_output = task_fact.output.raw
else:
    final_output = f"Unrecognized classification: {label}"

# Print the final output
print("Final Routed Output:\n", final_output)


Final Routed Output:
 <think>
Okay, so I need to figure out how to summarize the query "Explain how black holes form" into two concise sentences. Let me start by recalling what I know about black holes. From what I've learned, black holes are regions in space where the gravitational pull is so strong that nothing, not even light, can escape. They are formed when a massive star collapses in on itself.

Wait, so the process begins with a massive star. Stars typically fuse elements in their cores, which provides the outward pressure that balances the inward pull of gravity. When a massive star runs out of fuel, this balance is disrupted. Without the outward pressure, the star begins to collapse under its own gravity.

As the star collapses, the core becomes incredibly dense. I think this is where the concept of a singularity comes in—the center of the black hole where the density is infinite and the laws of physics as we know them break down. The point of no return around a black hole is 

In [21]:
# Section 5: Orchestrator-Worker – CrewAI using groq/deepseek-r1-distill-llama-70b
# Set Groq API credentials
import os
import time

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

# Import CrewAI
from crewai import Agent, Task, Crew, Process
from typing import List

# Define model to use
llm_model = "groq/deepseek-r1-distill-llama-70b"

# Define the Orchestrator agent: plans sections for a report
orchestrator_agent = Agent(
    name="OrchestratorAgent",
    role="Report Planner",
    goal="Plan sections for the report",
    backstory="The orchestrator generates a plan for the report with a title and description for each section.",
    llm=llm_model,
    tools=[],
    system_prompt="Create a plan for the report by dividing it into 2 sections, each with a title and a brief description.",
    verbose=True  # Enable verbose for debugging
)

# Define the worker agents: each one writes a section of the report
worker_agent = Agent(
    name="WorkerAgent",
    role="Section Writer",
    goal="Write a section of the report",
    backstory="The worker agent writes sections of the report based on the title and description provided by the orchestrator.",
    llm=llm_model,
    tools=[],
    system_prompt="Write the section '{title}' based on the description: {description}",
    verbose=True  # Enable verbose for debugging
)

# Define the synthesizer agent: merges all sections into the final report
synthesizer_agent = Agent(
    name="SynthesizerAgent",
    role="Report Synthesizer",
    goal="Combine all sections into one cohesive final report",
    backstory="The synthesizer combines all the sections into a coherent and well-written final report.",
    llm=llm_model,
    tools=[],
    system_prompt="Combine the following sections into a cohesive report.",
    verbose=True  # Enable verbose for debugging
)

# Define the task for the orchestrator: generates a report plan with titles and descriptions for each section
class SectionPlan:
    def __init__(self, title: str, description: str):
        self.title = title
        self.description = description

def generate_report_plan(state):
    sections = [
        SectionPlan("Introduction", "An introduction to the topic of AI."),
        SectionPlan("Applications of AI", "Discuss the various applications of AI in real-world scenarios."),
    ]
    print("Generated sections plan:", sections)  # Debug log
    return {"sections": sections}

# Define task for workers: writes a section of the report
def write_section(state):
    section = state["section"]
    print(f"Writing section: {section.title}")  # Debug log
    result = llm.invoke(f"Write a section titled '{section.title}': {section.description}")
    print(f"Section content: {result.content}")  # Debug log
    return {"section_content": result.content}

# Define the synthesizer task: combines all sections into the final report
def combine_sections(state):
    sections = state["sections"]
    final_report = "\n\n".join([f"### {section.title}\n{section.description}" for section in sections])
    print("Final report content:", final_report)  # Debug log
    return {"final_report": final_report}

# Define tasks
orchestrator_task = Task(
    name="OrchestratorTask",
    description="Plan the sections of the report.",
    expected_output="List of section titles and descriptions.",
    agent=orchestrator_agent
)

worker_task = Task(
    name="WorkerTask",
    description="Write a report section based on the plan.",
    expected_output="Section content.",
    agent=worker_agent
)

synthesizer_task = Task(
    name="SynthesizerTask",
    description="Combine all report sections into a final report.",
    expected_output="Final report.",
    agent=synthesizer_agent
)

# Create the crew: orchestrator, workers, and synthesizer
crew = Crew(
    agents=[orchestrator_agent, worker_agent, synthesizer_agent],
    tasks=[orchestrator_task, worker_task, synthesizer_task],
    process=Process.sequential,
    max_rounds=3
)

# Run the entire Orchestrator-Worker flow for report generation
state = crew.kickoff(inputs={})

# Debugging: Check what the state object contains after the tasks run
print("State after running tasks:", state)

# Access the final report (after all tasks are completed)
final_report = state["final_report"] if "final_report" in state else "Report generation failed."

# Add delay to avoid hitting rate limits
time.sleep(10)  # delay to handle Groq's rate limits

# Print the final report
print("Final Report:\n", final_report)

# Agent: Report Planner
## Task: Plan the sections of the report.


# Agent: Report Planner
## Final Answer: 
1. **Introduction**  
   - Purpose: Introduce the report's topic, explaining its significance and objectives.  
   - Scope: Outline the report's structure and what readers can expect.  

2. **Background Information**  
   - Context: Provide historical or contextual information relevant to the topic.  
   - Definitions: Define key terms or concepts necessary for understanding the report.  

3. **Analysis**  
   - Key Findings: Present the main data, research, or analysis that forms the core of the report.  
   - Supporting Evidence: Include charts, graphs, or examples that support the analysis.  

4. **Challenges and Limitations**  
   - Obstacles: Discuss any challenges faced during the research or analysis.  
   - Limitations: Acknowledge any limitations in the data or methodology.  

5. **Recommendations**  
   - Actionable Steps: Provide clear, practical recommendations base

In [22]:
# Section 6: Evaluator-Optimizer – CrewAI using groq/deepseek-r1-distill-llama-70b
# This section demonstrates a feedback loop where one agent generates a draft,
# another agent evaluates it, and if the feedback is negative, the draft is reworked.

# Set Groq API credentials
import os
import time

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

# Import CrewAI
from crewai import Agent, Task, Crew, Process

# Define model to use
llm_model = "groq/deepseek-r1-distill-llama-70b"

# Define the agent to generate tweets
generator_agent = Agent(
    name="GeneratorAgent",
    role="Tweet Generator",
    goal="Generate tweets based on the given topic",
    backstory="This agent drafts tweets based on given topics, but needs feedback to ensure quality.",
    llm=llm_model,
    tools=[],
    system_prompt="Write a tweet about the given topic. Be concise and engaging.",
    verbose=True  # Enable verbose logging for debugging
)

# Define the evaluator agent to provide feedback
evaluator_agent = Agent(
    name="EvaluatorAgent",
    role="Tweet Evaluator",
    goal="Evaluate the quality of the tweet and provide feedback",
    backstory="This agent judges the quality of the tweets and provides suggestions for improvement.",
    llm=llm_model,
    tools=[],
    system_prompt="Evaluate the following tweet and provide feedback on how to improve it. Respond with a grade (good/bad) and suggestions for improvement.",
    verbose=True  # Enable verbose logging for debugging
)

# Define the task for generating the initial tweet
tweet_task = Task(
    name="GenerateTweet",
    description="Generate a tweet about the given topic.",
    expected_output="Tweet text.",
    agent=generator_agent
)

# Define the task for evaluating the tweet
evaluate_task = Task(
    name="EvaluateTweet",
    description="Evaluate the generated tweet for quality and provide feedback.",
    expected_output="Evaluation text with grade and feedback.",
    agent=evaluator_agent
)

# Create the crew: generator agent, evaluator agent
crew = Crew(
    agents=[generator_agent, evaluator_agent],
    tasks=[tweet_task, evaluate_task],
    process=Process.sequential,
    max_rounds=3  # Max rounds for the loop (will stop after 3 iterations if needed)
)

# Run the evaluator-optimizer loop for tweet generation
topic = "AI in healthcare"
result = crew.kickoff(inputs={"topic": topic})

# Check the generated tweet and feedback
tweet = tweet_task.output.raw
feedback = evaluate_task.output.raw

# Add a sleep time to avoid rate limit issues
time.sleep(5)  # Adding a delay of 5 seconds to avoid hitting rate limits

# Check if the feedback is good or bad and decide whether to regenerate the tweet
if "bad" in feedback.lower():
    # If the feedback is bad, regenerate the tweet using the feedback
    print("Tweet feedback: Bad, regenerating tweet.")
    print("Original Tweet:", tweet)
    print("Feedback:", feedback)

    # Regenerate the tweet based on feedback (continue the feedback loop)
    result = crew.kickoff(inputs={"topic": topic})  # Retry with the same topic and feedback loop
else:
    # If the feedback is good, print the final tweet
    print("Tweet feedback: Good, final tweet:")
    print("Final Tweet:", tweet)
    print("Feedback:", feedback)


# Agent: Tweet Generator
## Task: Generate a tweet about the given topic.


# Agent: Tweet Generator
## Final Answer: 
"Reading is a journey through endless worlds! 📚✨ Escape reality and explore new ideas every day. #ReadingIsFun"


# Agent: Tweet Evaluator
## Task: Evaluate the generated tweet for quality and provide feedback.


# Agent: Tweet Evaluator
## Final Answer: 
<think>
Okay, so I need to evaluate this tweet: "Reading is a journey through endless worlds! 📚✨ Escape reality and explore new ideas every day. #ReadingIsFun". Let me break this down.

First, the content. The tweet is about reading, which is a positive and relatable topic. It uses a metaphor, comparing reading to a journey through endless worlds. That's creative and engaging. The use of emojis, specifically the book (📚) and sparkles (✨), adds a visual element that can catch attention on social media. 

Next, the structure. The tweet is concise, which is good because Twitter has a character limit. It's split into two 

In [23]:
# Section 7: ReAct Agent – CrewAI using groq/deepseek-r1-distill-llama-70b
# This section demonstrates a ReAct-style agent where the agent iterates over a task,
# calls tools, and adapts its output based on feedback until it reaches a valid solution.

# Set Groq API credentials
import os
import time

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

# Import CrewAI
from crewai import Agent, Task, Crew, Process

# Define model to use
llm_model = "groq/deepseek-r1-distill-llama-70b"

# Define the agent that generates tweets
generator_agent = Agent(
    name="GeneratorAgent",
    role="Tweet Generator",
    goal="Generate tweets based on the given topic",
    backstory="This agent drafts tweets based on given topics, but needs feedback to ensure quality.",
    llm=llm_model,
    tools=[],
    system_prompt="Write a tweet about the given topic. Be concise and engaging.",
    verbose=True  # Enable verbose logging for debugging
)

# Define the evaluator agent to provide feedback
evaluator_agent = Agent(
    name="EvaluatorAgent",
    role="Tweet Evaluator",
    goal="Evaluate the quality of the tweet and provide feedback",
    backstory="This agent judges the quality of the tweets and provides suggestions for improvement.",
    llm=llm_model,
    tools=[],
    system_prompt="Evaluate the following tweet and provide feedback on how to improve it. Respond with a grade (good/bad) and suggestions for improvement.",
    verbose=True  # Enable verbose logging for debugging
)

# Define the task for generating the initial tweet
tweet_task = Task(
    name="GenerateTweet",
    description="Generate a tweet about the given topic.",
    expected_output="Tweet text.",
    agent=generator_agent
)

# Define the task for evaluating the tweet
evaluate_task = Task(
    name="EvaluateTweet",
    description="Evaluate the generated tweet for quality and provide feedback.",
    expected_output="Evaluation text with grade and feedback.",
    agent=evaluator_agent
)

# Create the crew: generator agent, evaluator agent
crew = Crew(
    agents=[generator_agent, evaluator_agent],
    tasks=[tweet_task, evaluate_task],
    process=Process.sequential,
    max_rounds=3  # Max rounds for the loop (will stop after 3 iterations if needed)
)

# Run the evaluator-optimizer loop for tweet generation
topic = "AI in healthcare"
result = crew.kickoff(inputs={"topic": topic})

# Check the generated tweet and feedback
tweet = tweet_task.output.raw
feedback = evaluate_task.output.raw

# Add a sleep time to avoid rate limit issues
time.sleep(5)  # Adding a delay of 5 seconds to avoid hitting rate limits

# Check if the feedback is good or bad and decide whether to regenerate the tweet
if "bad" in feedback.lower():
    # If the feedback is bad, regenerate the tweet using the feedback
    print("Tweet feedback: Bad, regenerating tweet.")
    print("Original Tweet:", tweet)
    print("Feedback:", feedback)

    # Regenerate the tweet based on feedback (continue the feedback loop)
    result = crew.kickoff(inputs={"topic": topic})  # Retry with the same topic and feedback loop
else:
    # If the feedback is good, print the final tweet
    print("Tweet feedback: Good, final tweet:")
    print("Final Tweet:", tweet)
    print("Feedback:", feedback)


# Agent: Tweet Generator
## Task: Generate a tweet about the given topic.


# Agent: Tweet Generator
## Final Answer: 
<think>
Okay, I need to create a tweet based on the given topic. Let me think about the key points to include. The topic is about the importance of mental health, so I should emphasize taking care of one's mind. Maybe include something about seeking help without shame. Also, using a metaphor like a garden could make it more relatable. I should keep it positive and encouraging. Let me draft a few lines and make sure it's concise enough for a tweet.
</think>

Taking care of your mental health is just as important as your physical health. Remember, seeking help is a sign of strength, not weakness. Nurture your mind like a garden—water it with positivity, prune the negativity, and let it grow into something beautiful. #MentalHealthMatters


# Agent: Tweet Evaluator
## Task: Evaluate the generated tweet for quality and provide feedback.


# Agent: Tweet Evaluator
## Final A